In [1]:
from build_dataset_dataloader import get_dataset

train_dataset, val_dataset, vocabulary = get_dataset(case=1)

/home/thangquang09/CODE/CTAI_MachineLearning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Kích thước từ vựng: 9083
Index của '<pad>': 0
Index của một từ ngẫu nhiên 'hello': 2
Index của một từ không có trong từ điển: 2


In [2]:
len(vocabulary)

9083

In [3]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)

In [4]:
next(iter(train_dataloader))

[tensor([[1069,   13,   26,  ...,    0,    0,    0],
         [2220,  227,    4,  ..., 1376, 1781,  722]]),
 tensor([[ 261,   74,   32,  ...,    0,    0,    0],
         [2220,  227,    4,  ...,    0,    0,    0]]),
 tensor([1., 1.])]

In [5]:
import torch
from torch import nn
from LSTM import SiameseLSTM

EMBEDDING_DIM = 64
HIDDEN_DIM = 128
OUTPUT_DIM = 1 # Output 1 giá trị logit cho binary classification
NUM_EPOCHS = 20
LEARNING_RATE = 0.001
VOCAB_SIZE = len(vocabulary)
SEQ_LENGTH = 600

model = SiameseLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [12]:
from tqdm import tqdm
model.train()
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    total_correct = 0
    for seq1, seq2, labels in train_dataloader:
        seq1, seq2, labels = seq1.to(device), seq2.to(device), labels.to(device)

        # 1. Xóa gradient
        optimizer.zero_grad()

        # 2. Forward pass
        outputs = model(seq1, seq2)

        # 3. Tính loss
        # outputs có shape [batch_size, 1], cần squeeze để thành [batch_size]
        loss = criterion(outputs.squeeze(), labels)

        # 4. Backward pass
        loss.backward()

        # 5. Cập nhật trọng số
        optimizer.step()

        total_loss += loss.item()
        total_correct += outputs.argmax(dim=1).eq(labels).sum().item()
        break

    avg_loss = total_loss / len(train_dataloader)
    # if (epoch + 1) % 5 == 0:
    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {avg_loss:.4f}, Accuracy: {total_correct / len(train_dataset):.4f}')
    break



Epoch [1/20], Loss: 0.0093, Accuracy: 0.0068
